## Import modules

In [1]:
import sklearn
from sklearn.model_selection import train_test_split
import xgboost
import catboost
from catboost import CatBoostClassifier, Pool
import lightgbm
import imblearn
import lime
import seaborn
import mlxtend
import tsfresh
import rpy2
import graphviz
import pydot
import pandas as pd
import numpy as np
print("No errors, all good!")

No errors, all good!


## Load data
Load data from csv/txt files.

In [52]:
df_test = pd.read_csv("data/challenge2_test.csv")
# Uncomment line below to preview five first rows
#df_test.head()

df_train = pd.read_csv("data/challenge2_train.csv")
# Uncomment line below to preview five first rows
print(df_train.head())

df_demo = pd.read_csv("data/demo_prediction.txt")
# Uncomment line below to preview five first rows
#df_demo.head()

   id  target   f0  f1 f2   f3 f4    f5   f6   f7  ...  f19  f20  f21  f22  \
0   0       0  1.0  gL  e  3.0  A   NaN  0.0  6.0  ...  0.5  0.0  3.0    R   
1   1       0  0.0  Rj  c  1.0  A   7.0  1.0  4.0  ...  0.4  0.0  1.0  NaN   
2   2       0  NaN  In  a  1.0  A  10.0  1.0  6.0  ...  0.5  1.0  3.0    G   
3   3       1  1.0  rA  c  3.0  A   7.0  1.0  1.0  ...  0.6  1.0  1.0    G   
4   4       0  1.0  pE  c  3.0  A   7.0  0.0  6.0  ...  0.5  0.0  1.0    B   

         f23       f24 f25 f26  f27      f28  
0  328b0cf4e  0.834041   T   N  1.0  14.2364  
1  328b0cf4e  0.686021   T   N  1.0      NaN  
2  0c67fcbbd  1.141271   T   N  3.0      NaN  
3  fee4e3007  0.662382   T   N  3.0      NaN  
4  587e040bd -1.000000   T   N  1.0  13.9537  

[5 rows x 31 columns]


### Data processing
Convert from strings to best possible data types. 
Swap columns so that target is the final column.

In [51]:
def df_column_switch(df, column1, column2):
    i = list(df.columns)
    a, b = i.index(column1), i.index(column2)
    i[b], i[a] = i[a], i[b]
    df = df[i]
    return df

def get_sets(data):
    nums = data[numerical_columns].fillna(-999).convert_dtypes(False, True, False, False, False)
    cats = data[categorical_columns].fillna("-999")
    return pd.concat([nums,cats], axis=1)

numerical_columns = ["f0", "f3", "f5", "f6", "f7", "f11", "f16","f17","f19","f20","f21","f24", "f27", "f28"]
categorical_columns = ["f1", "f2", "f4", "f8", "f9", "f10", "f12", "f13", "f14", "f15", "f18", "f22", "f23", "f25", "f26"]
labels = numerical_columns + categorical_columns

df_train_2 = get_sets(df_train)
# df_train = df_column_switch(df_train, "target", "f28")
df_test_2 = get_sets(df_test)

print(df_train.head())

   id  target   f0  f1 f2   f3 f4    f5   f6   f7  ...  f19  f20  f21  f22  \
0   0       0  1.0  gL  e  3.0  A   NaN  0.0  6.0  ...  0.5  0.0  3.0    R   
1   1       0  0.0  Rj  c  1.0  A   7.0  1.0  4.0  ...  0.4  0.0  1.0  NaN   
2   2       0  NaN  In  a  1.0  A  10.0  1.0  6.0  ...  0.5  1.0  3.0    G   
3   3       1  1.0  rA  c  3.0  A   7.0  1.0  1.0  ...  0.6  1.0  1.0    G   
4   4       0  1.0  pE  c  3.0  A   7.0  0.0  6.0  ...  0.5  0.0  1.0    B   

         f23       f24 f25 f26  f27      f28  
0  328b0cf4e  0.834041   T   N  1.0  14.2364  
1  328b0cf4e  0.686021   T   N  1.0      NaN  
2  0c67fcbbd  1.141271   T   N  3.0      NaN  
3  fee4e3007  0.662382   T   N  3.0      NaN  
4  587e040bd -1.000000   T   N  1.0  13.9537  

[5 rows x 31 columns]


In [46]:
array = df_train.values
Y = df_train["target"]
X = array[:, 0:-1]

In [47]:
X_train, X_test, y_train, y_test = train_test_split(df_train_2, Y, train_size=0.8, random_state=42)

In [55]:
import numpy as np

from catboost import CatBoostClassifier, Pool

model = CatBoostClassifier(iterations=1000,
                           depth=16,
                           learning_rate=1,
                           loss_function='Logloss',
                           cat_features=categorical_columns,
                           verbose=True)
# train the model
model.fit(df_train_2, Y, cat_features=categorical_columns)
# make the prediction using the resulting model
preds_class = model.predict(df_test_2)
preds_proba = model.predict_proba(df_test_2)
print("class = ", preds_class)
print("proba = ", preds_proba)



0:	learn: 0.4443426	total: 7.2s	remaining: 1h 59m 51s
1:	learn: 0.3054212	total: 14s	remaining: 1h 56m 26s
2:	learn: 0.2354240	total: 20.7s	remaining: 1h 54m 30s
3:	learn: 0.1890604	total: 27.5s	remaining: 1h 54m 5s
4:	learn: 0.1366180	total: 34.1s	remaining: 1h 53m
5:	learn: 0.1046935	total: 41s	remaining: 1h 53m 17s
6:	learn: 0.0919495	total: 47.7s	remaining: 1h 52m 44s
7:	learn: 0.0756286	total: 54.7s	remaining: 1h 53m
8:	learn: 0.0630422	total: 1m 1s	remaining: 1h 53m 25s
9:	learn: 0.0546129	total: 1m 8s	remaining: 1h 53m 12s
10:	learn: 0.0487307	total: 1m 15s	remaining: 1h 53m 11s
11:	learn: 0.0402367	total: 1m 22s	remaining: 1h 53m 2s
12:	learn: 0.0352426	total: 1m 28s	remaining: 1h 52m 34s
13:	learn: 0.0312576	total: 1m 35s	remaining: 1h 51m 42s
14:	learn: 0.0278792	total: 1m 41s	remaining: 1h 51m 32s
15:	learn: 0.0252434	total: 1m 48s	remaining: 1h 51m 42s
16:	learn: 0.0231905	total: 1m 55s	remaining: 1h 51m 22s
17:	learn: 0.0219165	total: 2m 2s	remaining: 1h 51m 6s
18:	learn: 

144:	learn: 0.0016200	total: 15m 59s	remaining: 1h 34m 18s
145:	learn: 0.0016059	total: 16m 6s	remaining: 1h 34m 13s
146:	learn: 0.0015926	total: 16m 13s	remaining: 1h 34m 6s
147:	learn: 0.0015926	total: 16m 16s	remaining: 1h 33m 43s
148:	learn: 0.0015926	total: 16m 20s	remaining: 1h 33m 20s
149:	learn: 0.0015793	total: 16m 27s	remaining: 1h 33m 15s
150:	learn: 0.0015676	total: 16m 33s	remaining: 1h 33m 7s
151:	learn: 0.0015555	total: 16m 40s	remaining: 1h 33m
152:	learn: 0.0015436	total: 16m 47s	remaining: 1h 32m 59s
153:	learn: 0.0015436	total: 16m 51s	remaining: 1h 32m 39s
154:	learn: 0.0015314	total: 16m 59s	remaining: 1h 32m 39s
155:	learn: 0.0015314	total: 17m 3s	remaining: 1h 32m 19s
156:	learn: 0.0015192	total: 17m 12s	remaining: 1h 32m 23s
157:	learn: 0.0015075	total: 17m 19s	remaining: 1h 32m 20s
158:	learn: 0.0014968	total: 17m 27s	remaining: 1h 32m 18s
159:	learn: 0.0014968	total: 17m 30s	remaining: 1h 31m 56s
160:	learn: 0.0014968	total: 17m 36s	remaining: 1h 31m 43s
161:	

285:	learn: 0.0013238	total: 26m 54s	remaining: 1h 7m 39s
286:	learn: 0.0013238	total: 26m 58s	remaining: 1h 7m 29s
287:	learn: 0.0013238	total: 27m 3s	remaining: 1h 7m 21s
288:	learn: 0.0013238	total: 27m 7s	remaining: 1h 7m 11s
289:	learn: 0.0013238	total: 27m 10s	remaining: 1h 7m
290:	learn: 0.0013238	total: 27m 15s	remaining: 1h 6m 51s
291:	learn: 0.0013238	total: 27m 19s	remaining: 1h 6m 42s
292:	learn: 0.0013238	total: 27m 24s	remaining: 1h 6m 35s
293:	learn: 0.0013238	total: 27m 29s	remaining: 1h 6m 27s
294:	learn: 0.0013238	total: 27m 32s	remaining: 1h 6m 16s
295:	learn: 0.0013238	total: 27m 36s	remaining: 1h 6m 7s
296:	learn: 0.0013238	total: 27m 41s	remaining: 1h 5m 58s
297:	learn: 0.0013238	total: 27m 45s	remaining: 1h 5m 50s
298:	learn: 0.0013238	total: 27m 49s	remaining: 1h 5m 41s
299:	learn: 0.0013151	total: 27m 59s	remaining: 1h 5m 45s
300:	learn: 0.0013151	total: 28m 3s	remaining: 1h 5m 36s
301:	learn: 0.0013151	total: 28m 8s	remaining: 1h 5m 29s
302:	learn: 0.0013151	t

431:	learn: 0.0013062	total: 36m 37s	remaining: 48m 22s
432:	learn: 0.0013062	total: 36m 41s	remaining: 48m 15s
433:	learn: 0.0013062	total: 36m 44s	remaining: 48m 8s
434:	learn: 0.0013062	total: 36m 48s	remaining: 48m 1s
435:	learn: 0.0013062	total: 36m 53s	remaining: 47m 56s
436:	learn: 0.0013062	total: 36m 56s	remaining: 47m 49s
437:	learn: 0.0013062	total: 37m	remaining: 47m 42s
438:	learn: 0.0013062	total: 37m 4s	remaining: 47m 35s
439:	learn: 0.0013062	total: 37m 8s	remaining: 47m 29s
440:	learn: 0.0013062	total: 37m 12s	remaining: 47m 22s
441:	learn: 0.0013062	total: 37m 16s	remaining: 47m 16s
442:	learn: 0.0013062	total: 37m 22s	remaining: 47m 12s
443:	learn: 0.0013062	total: 37m 26s	remaining: 47m 5s
444:	learn: 0.0013062	total: 37m 29s	remaining: 46m 58s
445:	learn: 0.0013062	total: 37m 33s	remaining: 46m 52s
446:	learn: 0.0013062	total: 37m 37s	remaining: 46m 45s
447:	learn: 0.0013062	total: 37m 42s	remaining: 46m 39s
448:	learn: 0.0013062	total: 37m 45s	remaining: 46m 32s
4

579:	learn: 0.0012880	total: 46m 27s	remaining: 33m 45s
580:	learn: 0.0012880	total: 46m 32s	remaining: 33m 40s
581:	learn: 0.0012880	total: 46m 36s	remaining: 33m 35s
582:	learn: 0.0012880	total: 46m 40s	remaining: 33m 30s
583:	learn: 0.0012880	total: 46m 44s	remaining: 33m 24s
584:	learn: 0.0012880	total: 46m 48s	remaining: 33m 19s
585:	learn: 0.0012880	total: 46m 52s	remaining: 33m 13s
586:	learn: 0.0012880	total: 46m 56s	remaining: 33m 8s
587:	learn: 0.0012880	total: 47m	remaining: 33m 2s
588:	learn: 0.0012798	total: 47m 6s	remaining: 32m 59s
589:	learn: 0.0012798	total: 47m 10s	remaining: 32m 53s
590:	learn: 0.0012798	total: 47m 14s	remaining: 32m 48s
591:	learn: 0.0012798	total: 47m 19s	remaining: 32m 43s
592:	learn: 0.0012798	total: 47m 23s	remaining: 32m 37s
593:	learn: 0.0012798	total: 47m 26s	remaining: 32m 32s
594:	learn: 0.0012798	total: 47m 30s	remaining: 32m 26s
595:	learn: 0.0012798	total: 47m 35s	remaining: 32m 22s
596:	learn: 0.0012798	total: 47m 39s	remaining: 32m 16s

727:	learn: 0.0012711	total: 56m 26s	remaining: 21m 8s
728:	learn: 0.0012711	total: 56m 30s	remaining: 21m 4s
729:	learn: 0.0012711	total: 56m 34s	remaining: 20m 59s
730:	learn: 0.0012711	total: 56m 38s	remaining: 20m 54s
731:	learn: 0.0012711	total: 56m 42s	remaining: 20m 49s
732:	learn: 0.0012711	total: 56m 46s	remaining: 20m 44s
733:	learn: 0.0012711	total: 56m 50s	remaining: 20m 39s
734:	learn: 0.0012711	total: 56m 54s	remaining: 20m 34s
735:	learn: 0.0012711	total: 56m 58s	remaining: 20m 29s
736:	learn: 0.0012711	total: 57m 2s	remaining: 20m 24s
737:	learn: 0.0012711	total: 57m 5s	remaining: 20m 19s
738:	learn: 0.0012711	total: 57m 10s	remaining: 20m 14s
739:	learn: 0.0012711	total: 57m 14s	remaining: 20m 9s
740:	learn: 0.0012711	total: 57m 18s	remaining: 20m 4s
741:	learn: 0.0012711	total: 57m 22s	remaining: 20m
742:	learn: 0.0012711	total: 57m 26s	remaining: 19m 55s
743:	learn: 0.0012711	total: 57m 30s	remaining: 19m 50s
744:	learn: 0.0012711	total: 57m 33s	remaining: 19m 45s
74

872:	learn: 0.0012711	total: 1h 6m 3s	remaining: 9m 37s
873:	learn: 0.0012711	total: 1h 6m 6s	remaining: 9m 33s
874:	learn: 0.0012711	total: 1h 6m 10s	remaining: 9m 28s
875:	learn: 0.0012711	total: 1h 6m 14s	remaining: 9m 23s
876:	learn: 0.0012711	total: 1h 6m 17s	remaining: 9m 19s
877:	learn: 0.0012711	total: 1h 6m 21s	remaining: 9m 14s
878:	learn: 0.0012711	total: 1h 6m 25s	remaining: 9m 9s
879:	learn: 0.0012711	total: 1h 6m 29s	remaining: 9m 5s
880:	learn: 0.0012711	total: 1h 6m 32s	remaining: 9m
881:	learn: 0.0012711	total: 1h 6m 37s	remaining: 8m 55s
882:	learn: 0.0012711	total: 1h 6m 41s	remaining: 8m 51s
883:	learn: 0.0012711	total: 1h 6m 44s	remaining: 8m 46s
884:	learn: 0.0012711	total: 1h 6m 49s	remaining: 8m 42s
885:	learn: 0.0012711	total: 1h 6m 53s	remaining: 8m 37s
886:	learn: 0.0012711	total: 1h 6m 57s	remaining: 8m 32s
887:	learn: 0.0012711	total: 1h 7m	remaining: 8m 28s
888:	learn: 0.0012711	total: 1h 7m 5s	remaining: 8m 23s
889:	learn: 0.0012711	total: 1h 7m 9s	remain

In [56]:
stacked_columns = np.column_stack((df_test["id"], preds_proba[:, 1]))
df = pd.DataFrame(stacked_columns, columns=["id", "target"])
convert_dict={"id": int, "target" : float}
df = df.astype(convert_dict)
df.to_csv("predictions.csv", index = False)